# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,stanley,54.8680,-1.6985,17.38,58,20,4.12,GB,1683644734
1,1,port-aux-francais,-49.3500,70.2167,5.48,92,100,6.57,TF,1683644734
2,2,margaret river,-33.9500,115.0667,18.73,69,4,9.17,AU,1683644735
3,3,blackmans bay,-43.0167,147.3167,10.96,79,87,4.98,AU,1683644735
4,4,bethel,41.3712,-73.4140,18.43,37,0,2.06,US,1683644735


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    alpha = 0.5
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# A max temperature lower than 27 degrees but higher than 21
# Wind speed less than 4.5 m/s
# Zero cloudiness


# Narrow down cities that fit criteria and drop any results with null values
clean_df = city_data_df.copy()
min_temp = (clean_df['Max Temp'] >= 21)
max_temp = (clean_df['Max Temp'] <= 27)
wind_speed = (clean_df['Wind Speed'] <= 4.5)
cloudiness = (clean_df['Cloudiness'] == 0)

clean_df = clean_df.loc[min_temp & max_temp & wind_speed & cloudiness]


# Drop any rows with null values
clean_df = clean_df.dropna()

# Display sample data
clean_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
63,63,fortuna foothills,32.6578,-114.4119,24.56,31,0,3.60,US,1683644753
74,74,buka,40.8108,69.1986,22.48,31,0,3.14,UZ,1683644756
112,112,tansen,27.8673,83.5467,21.05,19,0,2.64,NP,1683644769
165,165,haiku-pauwela,20.9219,-156.3051,22.25,83,0,4.02,US,1683644788
282,282,diego de almagro,-26.3667,-70.0500,25.48,16,0,1.18,CL,1683644836
304,304,ixtapa,20.7000,-105.2000,25.57,53,0,0.00,MX,1683644843
346,346,larbaa,36.5647,3.1543,25.47,41,0,4.12,DZ,1683644857
438,438,villa union,23.2000,-106.2333,26.62,65,0,3.09,MX,1683644886
451,451,tura,25.5198,90.2201,26.29,32,0,2.36,IN,1683644890
481,481,gharavuti,37.5672,68.4469,24.36,23,0,1.71,TJ,1683644900


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_df[['City', 'Lat', 'Lng','Humidity', 'Country']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
63,fortuna foothills,32.6578,-114.4119,31,US,
74,buka,40.8108,69.1986,31,UZ,
112,tansen,27.8673,83.5467,19,NP,
165,haiku-pauwela,20.9219,-156.3051,83,US,
282,diego de almagro,-26.3667,-70.0500,16,CL,
304,ixtapa,20.7000,-105.2000,53,MX,
346,larbaa,36.5647,3.1543,41,DZ,
438,villa union,23.2000,-106.2333,65,MX,
451,tura,25.5198,90.2201,32,IN,
481,gharavuti,37.5672,68.4469,23,TJ,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories" : "accommodation.hotel",
    "limit" : 20,
    "apiKey" : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    long = row["Lng"]
    lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{long},{lat},{radius}'
    params["bias"] = f'proximity:{long},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
fortuna foothills - nearest hotel: Comfort Inn
buka - nearest hotel: No hotel found
tansen - nearest hotel: City View Homestay
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
diego de almagro - nearest hotel: Residencial Hambyll
ixtapa - nearest hotel: Ma. Cristina
larbaa - nearest hotel: No hotel found
villa union - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
gharavuti - nearest hotel: No hotel found
homestead meadows north - nearest hotel: No hotel found
hermosillo - nearest hotel: Lucerna
del rio - nearest hotel: La Quinta Inn by Wyndham Del Rio
tshabong - nearest hotel: No hotel found


,City,Lat,Lng,Humidity,Country,Hotel Name
63,fortuna foothills,32.6578,-114.4119,31,US,Comfort Inn
74,buka,40.8108,69.1986,31,UZ,No hotel found
112,tansen,27.8673,83.5467,19,NP,City View Homestay
165,haiku-pauwela,20.9219,-156.3051,83,US,Inn At Mama's Fish House
282,diego de almagro,-26.3667,-70.0500,16,CL,Residencial Hambyll
304,ixtapa,20.7000,-105.2000,53,MX,Ma. Cristina
346,larbaa,36.5647,3.1543,41,DZ,No hotel found
438,villa union,23.2000,-106.2333,65,MX,No hotel found
451,tura,25.5198,90.2201,32,IN,No hotel found
481,gharavuti,37.5672,68.4469,23,TJ,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    alpha = 0.5,
    hover_cols = ['Hotel Name','Country']
)

# Display the map
hotel_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)

In [14]:
%%capture --no-display
#Extra Map showing excluding results with no hotels
clean_hotel_df = hotel_df.loc[hotel_df['Hotel Name']!='No hotel found']


# Configure the map plot
clean_hotel_plot = clean_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    alpha = 0.5,
    hover_cols = ['Hotel Name','Country']
)

# Display the map
clean_hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)